In [ ]:
# ==========================================================
# RECLASSIFICAÇÃO AUTOMÁTICA – RESULTADO DO TAF
# Concurso PMCE / CBMCE
# ----------------------------------------------------------
# Este script:
# 1) Lê a classificação original (Excel da banca)
# 2) Lê o PDF do Resultado do TAF
# 3) Mantém apenas candidatos com SITUAÇÃO = APTO
# 4) Recalcula toda a classificação seguindo a lógica da banca
# 5) Gera um novo arquivo Excel com a classificação atualizada
# ==========================================================

# ==========================================================
# INSTALAÇÃO DAS BIBLIOTECAS (Google Colab)
# ==========================================================
!pip install pdfplumber pandas openpyxl

import pandas as pd
import pdfplumber
from google.colab import files

# ==========================================================
# 1) ENVIO DOS ARQUIVOS
# ==========================================================
print("📂 Envie o arquivo Excel da classificação original da banca:")
uploaded_excel = files.upload()
excel_path = list(uploaded_excel.keys())[0]

print("\n📂 Agora envie o PDF do Resultado do TAF:")
uploaded_pdf = files.upload()
pdf_path = list(uploaded_pdf.keys())[0]

# ==========================================================
# 2) LEITURA DA CLASSIFICAÇÃO ORIGINAL
# ==========================================================
df_base = pd.read_excel(excel_path)

# Garantir que as colunas numéricas estejam no formato correto
df_base["Pedido"] = pd.to_numeric(df_base["Pedido"], errors="coerce")
df_base["Nota"] = pd.to_numeric(df_base["Nota"], errors="coerce")

# Colunas de ordenação usadas pela banca
colunas_ordenacao = [
    "Ordenação Geral (Todos)",
    "Ordenação Geral (Masc)",
    "Ordenação Geral (Fem)",
    "Ordenação Especial (Todos)",
    "Ordenação Especial (Masc)",
    "Ordenação Especial (Fem)"
]

# Se alguma coluna não existir no Excel, ela é criada vazia
for col in colunas_ordenacao:
    if col not in df_base.columns:
        df_base[col] = pd.NA
    df_base[col] = pd.to_numeric(df_base[col], errors="coerce")

# ==========================================================
# 3) EXTRAÇÃO DOS CANDIDATOS APTOS NO TAF
# ==========================================================
def extrair_pedidos_aptos(pdf_path):
    """
    Lê o PDF do TAF e retorna uma lista com os números de pedido
    dos candidatos cuja SITUAÇÃO final seja 'Apto'.

    Observação:
    Mesmo que o candidato tenha sido inapto em algum exercício,
    ele será mantido se a situação final for 'Apto'.
    """

    pedidos = []

    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            texto = pagina.extract_text()
            if not texto:
                continue

            for linha in texto.split("\n"):
                partes = linha.split()

                # Linhas válidas começam com o número do pedido
                if partes and partes[0].isdigit():
                    pedido = int(partes[0])

                    # A última coluna do PDF é a SITUAÇÃO
                    situacao_final = partes[-1].strip().lower()

                    if situacao_final == "apto":
                        pedidos.append(pedido)

    # Remove duplicados preservando ordem
    return list(dict.fromkeys(pedidos))


pedidos_aptos = extrair_pedidos_aptos(pdf_path)

print(f"\n🔎 Total de candidatos APTOS encontrados no PDF: {len(pedidos_aptos)}")

# Filtrar apenas candidatos aptos no TAF
df = df_base[df_base["Pedido"].isin(pedidos_aptos)].copy()

print(f"📊 Candidatos localizados no Excel e mantidos na classificação: {len(df)}")

# ==========================================================
# 4) IDENTIFICAÇÃO DO SEXO (conforme estrutura da banca)
# ==========================================================
def deduzir_sexo(row):
    """
    O sexo é identificado verificando em quais colunas
    de ordenação o candidato possui valor.
    """

    if pd.notna(row["Ordenação Geral (Masc)"]):
        return "M"
    if pd.notna(row["Ordenação Geral (Fem)"]):
        return "F"
    if pd.notna(row["Ordenação Especial (Masc)"]):
        return "M"
    if pd.notna(row["Ordenação Especial (Fem)"]):
        return "F"

    return pd.NA


df["Sexo"] = df.apply(deduzir_sexo, axis=1)

# ==========================================================
# 5) FUNÇÃO PADRÃO DE ORDENAÇÃO
# ==========================================================
def ordenar(df_local, coluna_ordenacao):
    """
    Se a coluna oficial de ordenação existir,
    utiliza a ordenação da banca.
    Caso contrário, ordena pela Nota (maior para menor).
    """

    if df_local[coluna_ordenacao].notna().any():
        return df_local.sort_values(
            by=coluna_ordenacao,
            ascending=True,
            na_position="last"
        )

    return df_local.sort_values(
        by="Nota",
        ascending=False,
        na_position="last"
    )

# ==========================================================
# 6) RECLASSIFICAÇÃO – SEGUINDO A LÓGICA DA BANCA
# ==========================================================

# -------------------------
# GERAL – TODOS
# -------------------------
df = ordenar(df, "Ordenação Geral (Todos)").reset_index(drop=True)
df["Nova Ordenação Geral (Todos)"] = df.index + 1

# -------------------------
# GERAL – AMPLA (Masculino)
# -------------------------
df_m = df[
    (df["Sexo"] == "M") &
    (df["Segmento"].str.lower() == "ampla")
].copy()

df_m = ordenar(df_m, "Ordenação Geral (Todos)").reset_index(drop=True)
df_m["Nova Ordenação Geral (Masc)"] = df_m.index + 1

# -------------------------
# GERAL – AMPLA (Feminino)
# -------------------------
df_f = df[
    (df["Sexo"] == "F") &
    (df["Segmento"].str.lower() == "ampla")
].copy()

df_f = ordenar(df_f, "Ordenação Geral (Todos)").reset_index(drop=True)
df_f["Nova Ordenação Geral (Fem)"] = df_f.index + 1

# Inserir no dataframe principal
df = df.merge(df_m[["Pedido", "Nova Ordenação Geral (Masc)"]], on="Pedido", how="left")
df = df.merge(df_f[["Pedido", "Nova Ordenação Geral (Fem)"]], on="Pedido", how="left")

# -------------------------
# ESPECIAL – TODOS (cotistas)
# -------------------------
df_esp = df[df["Segmento"].str.lower() != "ampla"].copy()

df_esp = ordenar(df_esp, "Ordenação Especial (Todos)").reset_index(drop=True)
df_esp["Nova Ordenação Especial (Todos)"] = df_esp.index + 1

# Especial Masculino
df_esp_m = df_esp[df_esp["Sexo"] == "M"].copy()
df_esp_m = ordenar(df_esp_m, "Ordenação Especial (Todos)").reset_index(drop=True)
df_esp_m["Nova Ordenação Especial (Masc)"] = df_esp_m.index + 1

# Especial Feminino
df_esp_f = df_esp[df_esp["Sexo"] == "F"].copy()
df_esp_f = ordenar(df_esp_f, "Ordenação Especial (Todos)").reset_index(drop=True)
df_esp_f["Nova Ordenação Especial (Fem)"] = df_esp_f.index + 1

# Inserir no dataframe principal
df = df.merge(df_esp[["Pedido", "Nova Ordenação Especial (Todos)"]], on="Pedido", how="left")
df = df.merge(df_esp_m[["Pedido", "Nova Ordenação Especial (Masc)"]], on="Pedido", how="left")
df = df.merge(df_esp_f[["Pedido", "Nova Ordenação Especial (Fem)"]], on="Pedido", how="left")

# ==========================================================
# 7) ORGANIZAÇÃO FINAL DA TABELA
# ==========================================================
colunas_saida = [
    "Pedido",
    "Nome",
    "Segmento",
    "Nota",
    "Sexo",
    "Nova Ordenação Geral (Todos)",
    "Nova Ordenação Geral (Masc)",
    "Nova Ordenação Geral (Fem)",
    "Nova Ordenação Especial (Todos)",
    "Nova Ordenação Especial (Masc)",
    "Nova Ordenação Especial (Fem)"
]

df_final = df[colunas_saida].sort_values(
    by="Nova Ordenação Geral (Todos)"
)

# ==========================================================
# 8) RESUMO ESTATÍSTICO
# ==========================================================
df_resumo = pd.DataFrame({
    "Indicador": [
        "Total de candidatos aptos",
        "Total Masculino",
        "Total Feminino",
        "Total Segmento Especial",
        "Total Ampla",
        "Total de pedidos capturados no PDF"
    ],
    "Valor": [
        len(df_final),
        (df_final["Sexo"] == "M").sum(),
        (df_final["Sexo"] == "F").sum(),
        (df_final["Segmento"].str.lower() != "ampla").sum(),
        (df_final["Segmento"].str.lower() == "ampla").sum(),
        len(pedidos_aptos)
    ]
})

# ==========================================================
# 9) EXPORTAÇÃO DO ARQUIVO FINAL
# ==========================================================
arquivo_saida = "classificacao_recalculada_taf.xlsx"

with pd.ExcelWriter(arquivo_saida, engine="openpyxl") as writer:
    df_final.to_excel(writer, sheet_name="Classificação", index=False)
    df_resumo.to_excel(writer, sheet_name="Resumo", index=False)

print(f"\n💾 Arquivo gerado com sucesso: {arquivo_saida}")
files.download(arquivo_saida)